In [1]:
from dotenv import load_dotenv
import openai
import os
from llama_index.core import (
    Settings,
    VectorStoreIndex,
    SimpleDirectoryReader, 
    StorageContext,
    load_index_from_storage,
    Document,
    PromptTemplate
)
load_dotenv()

True

In [2]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.openai import OpenAI
llm = OpenAI(model="gpt-4o-mini")
embed_model = HuggingFaceEmbedding(model_name="intfloat/multilingual-e5-base")
Settings.embed_model = embed_model
Settings.llm = llm

In [3]:
def get_artist_documents(filename: str) -> list[Document]:
    with open(filename) as file:
        data = file.read()
    songs = data.split("===")
    artist = songs.pop(0).strip()
    
    documents = [
        Document(
            text=song,
            metadata={
                "category":"music",
                "artist": artist,
            }
        )
        for song in songs
    ]    
    return documents

In [4]:
import os
PERSIST_DIR = "lyrics_store2"

if not os.path.exists(PERSIST_DIR):
    documents = get_artist_documents("8988_Kjarkas.txt")
    index = VectorStoreIndex.from_documents(documents, show_progress=True)
    index.storage_context.persist(persist_dir=PERSIST_DIR)
else:
    storage_context = StorageContext.from_defaults(persist_dir=PERSIST_DIR) 
    index = load_index_from_storage(storage_context)

In [5]:
query_engine = index.as_query_engine()

In [6]:
qa_template_str = """
    You are an expert in Bolivian Folk music, your task is to guide and teach the user 
    about your field. Answer the user queries only with supported data in your context.
    Your context may contain complete lyrics or parts of them in different languages, but
    your answer will always be in Spanish. 

    Context information is below.
    ---------------------
    {context_str}
    ---------------------
    Given the context information and not prior knowledge, 
    answer the query with detailed source information, include direct quotes and use bullet lists in your 
    answers, in one of the bullets detail the tone/sentiment of the song.
    Query: {query_str}
    Answer: 
"""
qa_template = PromptTemplate(qa_template_str)

In [7]:
query_engine.update_prompts(
    {"response_synthesizer:text_qa_template": qa_template}
)

In [16]:
import wikipediaapi

def get_artist_wikipedia_info(artist: str) -> str:
    """
        Useful for getting artist information from wikipedia
        Use plain text as input to the tool, this should be the name of an artist or a band.
        The output of this tool is the summary of the wikipedia page corresponding to the artist.
    """
    wiki = wikipediaapi.Wikipedia("Test Wikipedia (eduardo.laruta@gmail.com)", "es")
    page = wiki.page(artist)
    if not page.exists():
        return f"Wikipedia page for {artist} does not exist"
    # print("====")
    # print(page.sections[0])
    # print("====")
    return f"{page.summary}\n{page.sections[0]}\n{page.sections[1]}"

display(Markdown(get_artist_wikipedia_info("tupay")))

Tupay es una agrupación musical folclórica de Bolivia formada en 1996 en Cochabamba por Edwin Castellanos y Fernando Torrico, quienes integraron por más de 10 años el grupo Los Kjarkas para después formar Pacha.
Actualmente la agrupación cuenta con Iver Villarroel como la primera voz del grupo y a Edwin Castellanos como director del grupo. 
Entre sus canciones más conocidas se encuentran "Ay, amor", "Ay, no sé qué tienes", "América Latina", "Mal acostumbrado", "La saya", "Soy caporal" y en su último disco tenemos "Cholita" y "Saya Reina". La palabra "tupay" significa "punto de encuentro" en quechua.
Section: Biografía (1):
Tanto Edwin Castellanos como Fernando Torrico ingresaron de muy jóvenes en la escuela musical de Los Kjarkas a principios de los 80, formando parte de Proyección-Kjarkas, un conjunto que más adelante derivaría en Proyección. Aparte de esto, Fernando Torrico ya había grabado algún trabajo en solitario y formó parte del grupo Fortaleza junto con Ramiro de la Zerda.
Esta temprana experiencia resultó básica en el perfeccionamiento artístico para nuestros dos artistas, que además les sirvió como trampolín para integrar el legendario grupo Los Kjarkas durante buena parte de los años 80 y principios de los 90, para muchos, la mejor formación que ha tenido Los Kjarkas en toda su dilatada historia.
Es precisamente en 1995 cuando Edwin y Fernando forman el grupo Pacha junto con otros dos integrantes de Los Kjarkas: Élmer Hermosa y Gastón Guardia. Pero tras grabar un solo disco, se separan definitivamente de Los Kjarkas y constituyen el dúo Tupay, marchan a Los Ángeles, EE. UU., y graban su primer trabajo bajo la dirección de Bebu Silvetti: América Latina.Llegaron a participar en la grabación del disco De mi alma latina II de nada más y nada menos que Plácido Domingo.
En 2006, Fernando Torrico abandona el dúo Tupay y es reemplazado por Rimer Guachalla grabando el disco Con Corazón último disco conocido del Dúo.
En 2018, debuta como su primera voz el cantante orureño, Iver Villarroel.
Subsections (0):

Section: Discografía (1):
1996 Música de toda la vida (disco grabado con el nombre de los integrantes, aún no como Tupay)
1997 América Latina
1998 Buscando Paz
1998 Folklore
2000 Carnaval - Coplas del nuevo milenio
2000 Sin fronteras
2000 Así es Bolivia
2001 Antología de la cueca
2002 Folklore con vida
2003 Colección de música en tu casa
2003 Viernes de soltero
2005 Tupay
2005 Love songs of América Latina
2006 Con corazón
2017 Rosas
2018 Cuecas con Bandoneón
2024 Folklore Con Altura
Subsections (0):


In [17]:
from llama_index.agent.openai import OpenAIAgent
from llama_index.core.tools import QueryEngineTool, FunctionTool, ToolMetadata

description = """
A set of lyrics for songs from the Bolivian Folk Group Los Kjarkas. 
Use plain text question as input to the tool. 
MANDATORY: Pass the response to the user as is, mantaining the format, do not try to summarize when using this tool.
"""
tools = [
    QueryEngineTool(
        query_engine=query_engine,
        metadata=ToolMetadata(
            name="Kjarkas_songs_lyrics",
            description=description,
            return_direct=False
        )
    ),
    FunctionTool.from_defaults(get_artist_wikipedia_info)
]


In [18]:
agent = OpenAIAgent.from_tools(tools=tools, verbose=True)

In [19]:
response = agent.chat("que canciones de los kjarkas hablan de traicion?")

Added user message to memory: que canciones de los kjarkas hablan de traicion?
=== Calling Function ===
Calling function: Kjarkas_songs_lyrics with args: {"input":"canciones de los kjarkas que hablan de traicion"}
Got output: Las canciones de Kjarkas que abordan el tema de la traición incluyen:

- **"Mi pecado"**
  - Esta canción trata sobre el dolor y la decepción que siente el narrador tras haber sido engañado por el amor. 
  - Fragmentos relevantes:
    - "Mi pecado fue quererte y haber caído en tu trampa"
    - "Víctima de tus caprichos, hoy he visto claramente"
    - "Yo creí en tus palabras sin saber que me engañabas"
  - **Tono/Sentimiento**: La canción tiene un tono melancólico y reflexivo, expresando tristeza y resignación ante la traición.

- **"Tiempo Al Tiempo"**
  - Aunque no se centra exclusivamente en la traición, refleja el dolor causado por las palabras de alguien que ha decepcionado al narrador.
  - Fragmentos relevantes:
    - "Me duelen tus palabras con que mastas m

In [20]:
from IPython.display import Markdown, display
display(Markdown(response.response))

Las canciones de Kjarkas que abordan el tema de la traición incluyen:

- **"Mi pecado"**
  - Esta canción trata sobre el dolor y la decepción que siente el narrador tras haber sido engañado por el amor. 
  - Fragmentos relevantes:
    - "Mi pecado fue quererte y haber caído en tu trampa"
    - "Víctima de tus caprichos, hoy he visto claramente"
    - "Yo creí en tus palabras sin saber que me engañabas"
  - **Tono/Sentimiento**: La canción tiene un tono melancólico y reflexivo, expresando tristeza y resignación ante la traición.

- **"Tiempo Al Tiempo"**
  - Aunque no se centra exclusivamente en la traición, refleja el dolor causado por las palabras de alguien que ha decepcionado al narrador.
  - Fragmentos relevantes:
    - "Me duelen tus palabras con que mastas mis ilusiones"
    - "Le daremos tiempo al tiempo, él nos dirá qué sucede"
  - **Tono/Sentimiento**: La canción transmite un sentimiento de tristeza y anhelo, con un enfoque en la esperanza de que el tiempo cure las heridas.

Estas canciones son ejemplos claros de cómo Kjarkas explora el tema de la traición y el dolor emocional en sus letras.

In [21]:
response = agent.chat("y quienes son ellos?")

Added user message to memory: y quienes son ellos?
=== Calling Function ===
Calling function: get_artist_wikipedia_info with args: {"artist":"Los Kjarkas"}
Got output: Los Kjarkas  es un conjunto de música folklórica originado en Capinota, Cochabamba, Bolivia. Fue formado por los hermanos Wilson Hermosa,​ Gonzalo Hermosa, Castel Hermosa​ y Edgar Villaroel. Alcanzaron popularidad nacional e internacional. Los compositores del conjunto fueron: Ulises Hermosa,​ Élmer Hermosa, Edgar Villaroel, Edwin Castellanos, Fernando Torrico, Gastón Guardia y Guillermo Ponce.
Section: Origen del nombre (1):
El nombre “Kjarkas” proviene del Quechua “Valiente,” o sea que la significación del nombre Los Kjarkas, sería: "Los Valientes" o "Los con Coraje."
Subsections (0):

Section: Historia (1):
En principio, Los Kjarkas surgió, debido a una necesidad económica de sus integrantes, por lo cual el grupo iba de pueblo en pueblo, tocando principalmente zambas argentinas, pues la música autóctona de Bolivia aún

In [22]:
display(Markdown(response.response))


Los Kjarkas es un conjunto de música folklórica originado en Capinota, Cochabamba, Bolivia. Fue formado por los hermanos Wilson Hermosa, Gonzalo Hermosa, Castel Hermosa y Edgar Villaroel. Alcanzaron popularidad tanto a nivel nacional como internacional. Los compositores del grupo incluyen a Ulises Hermosa, Élmer Hermosa, Edgar Villaroel, Edwin Castellanos, Fernando Torrico, Gastón Guardia y Guillermo Ponce.

El nombre “Kjarkas” proviene del Quechua y significa "Valiente", lo que se traduce como "Los Valientes" o "Los con Coraje."

### Historia
Los Kjarkas surgieron inicialmente por una necesidad económica, tocando en diferentes pueblos y comenzando con zambas argentinas. Con el tiempo, comenzaron a incorporar ritmos autóctonos de Bolivia, como la cueca y el huayño. A lo largo de los años, el grupo ha experimentado varios cambios en su formación y ha evolucionado en su estilo musical, adaptando ritmos autóctonos a estructuras musicales más modernas.

En 1976, publicaron su primer álbum titulado “Bolivia”, que incluye la famosa canción "Bolivia". A partir de ahí, su popularidad creció, organizando giras no solo por Bolivia, sino también por Sudamérica, Europa, Estados Unidos y Japón.

A lo largo de su carrera, han lanzado numerosos álbumes y han creado canciones icónicas como "Llorando se fue" y "Wayayay". A pesar de los desafíos, incluyendo la pérdida de algunos de sus miembros, Los Kjarkas han mantenido su legado y continúan siendo una de las agrupaciones más representativas de la música folklórica boliviana.

In [24]:
display(Markdown(agent.chat("y cual es su cancion mas popular?").response))

Added user message to memory: y cual es su cancion mas popular?


Una de las canciones más populares de Los Kjarkas es **"Llorando se fue"**. Esta canción se ha convertido en un clásico de la música folklórica boliviana y ha alcanzado reconocimiento internacional. La melodía y la letra emotiva han resonado con muchas personas, y la canción ha sido versionada por varios artistas en diferentes géneros. Además, su popularidad se vio incrementada cuando fue plagiada por el grupo Kaoma en la famosa canción "Lambada", lo que llevó a Los Kjarkas a recibir una indemnización. "Llorando se fue" sigue siendo un símbolo del legado musical de Los Kjarkas.